In [7]:
#This notebook is used to test the NN_model
#to get this notebook to work, have to manually set modul_path and scalers_path to be correct
#for the package which should work automatically, see instead SKiNN_package_example.ipynb
import os
import sys
module_path = os.path.abspath(os.path.join('/local/home/lbiggio/lensing_odyssey_kinematics/SKiNN/src'))
scalers_path = os.path.abspath(os.path.join('/local/home/lbiggio/lensing_odissey_kinematics/SKiNN/scalers/'))
if module_path not in sys.path:
    sys.path.append(module_path)
if scalers_path not in sys.path:
    sys.path.append(scalers_path)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="4"

from NN_models import*
import torch
from torch.utils.data import DataLoader, random_split
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
import joblib
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'NN_models'

### Import Model weights

In [2]:
weights_path = '/local/home/lbiggio/lensing_odyssey_kinematics/SKiNN/weights/upsampling_cosmo_gen_norm_1channel_new-epoch=1109-valid_loss=0.00.ckpt'


model=Generator_imp()
net = model.load_from_checkpoint(weights_path).cuda()

### Import Data

In [3]:
#trainx = np.load('epfldata/parameters_new.npy')
#trainy = np.load('epfldata/vrms_map_new.npy')

#scaling_y = np.load('../scalers/scaler_y.npy')
#scaling_x = joblib.load('../scalers/scaler_x')


trainx = np.load('/local/home/lbiggio/lensing_odyssey_kinematics/data/parameters1.npy')
trainy = np.load('/local/home/lbiggio/lensing_odyssey_kinematics/data/maps1.npy')
testx = np.load('/local/home/lbiggio/lensing_odyssey_kinematics/data/parameters2.npy')
testy = np.load('/local/home/lbiggio/lensing_odyssey_kinematics/data/maps2.npy')

testy=testy/trainy.max()
trainy=trainy/trainy.max()
min_max_scaler = preprocessing.MinMaxScaler()
scaler = min_max_scaler.fit(trainx)
testx = min_max_scaler.transform(testx)
trainx = min_max_scaler.transform(trainx)

train_dataset = TensorDataset(torch.Tensor(trainx),torch.Tensor(trainy))
test = TensorDataset(torch.Tensor(testx),torch.Tensor(testy))

### Interface

In [4]:
def generate_maps(net,scaling_x,scaling_y,input_p, plot=True):
    """Generate velocity maps given input parameters.
    
    net: neural network object
    scaling_x: input scaler
    scaling_y: output scaling
    input_p: input parameters
    plt: plot if True
    
    Returns the velocity maps 
    """
    
    input_p = scaling_x.transform(np.reshape(input_p,(-1,len(input_p))))
    input_p = torch.Tensor(input_p).cuda()
    net.eval()
    
    with torch.no_grad():
        pred = net(input_p.unsqueeze(0)).cpu().numpy()
    
    if plot:
        plt.figure(figsize=(24, 6))
        plt.subplot(131)
        plt.imshow(pred[0,:,:].squeeze())
        plt.title('Prediction')
        plt.colorbar()
    
    return pred*scaling_y #### rotation



def generate_maps_no_scaling(net,input_p):
    """Generate velocity maps given input parameters.
    
    net: neural network object
    scaling_x: input scaler
    scaling_y: output scaling
    input_p: input parameters
    plt: plot if True
    
    Returns the velocity maps 
    """
    
    input_p = torch.Tensor(input_p)
    net.eval()
    
    with torch.no_grad():
        pred = net(input_p.cuda()).cpu().numpy()
    
    return pred 

### Example

In [5]:
n = np.random.randint()
example_input = test.tensors[0][n,:].unsqueeze(0)
print('input params: ',example_input)
#output = generate_maps(net,scaling_x,scaling_y,example_input, plot=True)
plt.figure(figsize=(24, 6))
pred = generate_maps_no_scaling(net,example_input, plot=True)
plt.imshow(pred.numpy())

#plt.figure()
#test.tensors[1][n,:276,:276]

TypeError: randint() takes at least 1 positional argument (0 given)

In [ ]:
#span = np.linspace(-0.4,0.6,100)
#inp = example_input
#for s in span:
#    inp[-2] = s
#    output = generate_maps(net,scaling_x,scaling_y,inp, plot=True)
#    plt.title(str(s))